## TODO
class vehicle movement predictor
put vehicle to x0,y0,a0 and rotate all the points
receive vehicle north and host point


In [1]:
print('test')

test


## Cameras

In [2]:
from src.vision import Camera

mba_video_camera = Camera(
    h_fov=59.0,
    h_res=1080,
    v_res=720,
)

mba_photo_camera = Camera(
    h_fov=67.0,
    h_res=1290,
    v_res=720,
)

zed_two_camera = Camera(
    h_fov=110.0,
    v_fov=70.0,
)

avg_shoulder_width = 350.0
avg_torso_height = 460.0


In [3]:
from src.point import Point

ref_points = [
    Point(0, 0),
    Point(1200, 0),
    Point(2200, 0),
    Point(4200, 0),
    Point(3000, 1280),
    Point(3995, 1590),
    Point(3005, -1150),
    Point(4000, -1490),
]
print(ref_points)


[Point(x=0, y=0, a=None), Point(x=1200, y=0, a=None), Point(x=2200, y=0, a=None), Point(x=4200, y=0, a=None), Point(x=3000, y=1280, a=None), Point(x=3995, y=1590, a=None), Point(x=3005, y=-1150, a=None), Point(x=4000, y=-1490, a=None)]


In [4]:
from IPython.display import clear_output

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import cv2
import time

import torch



## YOLOv8 For human and objects detection

First attempt at human positioning with YOLOv8. Unsure if will be usefull because of the RGBD camera. It lets us estimates the angle between the camera and the person. Could also be usefull for object detection and avoidance. 

In [6]:
from src.vision import PositionEstimator, VideoCapture
import mediapipe as mp
from src.vision import VideoCapture
from src.vision import ObjectDetector
from src.vision import plot_obj
from ultralytics import YOLO
from src.vision import ReID
import time

verbose = False
refresh_rate = 1
camera = mba_video_camera
yolo_model = YOLO('yolov8s.pt')
obj_detector = ObjectDetector(yolo_model, camera)
pe = PositionEstimator(
    mp.solutions.pose, 
    camera,
    avg_torso_height, 
    file_path='pe.csv',
    queue_size=3,
    kernel_size=3
)

try:
    # Initiate the camera
    # cap = VideoCapture('./Movie on 2023-11-23 at 10.03.mov')
    # cap = VideoCapture('./Movie on 2023-12-11 at 21.01.mov')
    # cap = VideoCapture('./Movie on 2023-12-11 at 21.02.mov')
    cap = VideoCapture(0)
    time.sleep(1)

    # cap.skip_frames(14)
    frame = cap.grab_frame()
    obj_detector.detect(frame)
    host = obj_detector.get_objs_by_name('person', score=0.4).iloc[0]
    reid = ReID(frame, host, 32, 0.80)

    while True:
        start = time.time()
        # cap.skip_frames(5)
        frame = cap.grab_frame()
        if frame is None: break
        
        obj_detector.detect(frame)
        objs = obj_detector.get_objs_by_name('person', score=0.4)
        host, objs = reid.reid(frame, objs)

        clear_output(wait=True)
        fig, ax = plt.subplots() if verbose else (None, None)

        if host is not None:
            bo_frame = PositionEstimator.mask_frame(frame, host)
            result = pe.save_position(bo_frame, 1.15, verbose=verbose)
            if result is not None and verbose:
                print('result')
                pe.plot_result(ax, frame, result, camera)
        else:
            pe.save_blank()

        ax.imshow(frame) if verbose else None
        # ax.imshow(bo_frame) if verbose else None

        if verbose:
            plot_obj(ax, host, 'r') if host is not None else None
            for index, obj in objs.iterrows():
                plot_obj(ax, obj, 'y')
            plt.axis('off')
            plt.show()

        delta = time.time() - start
        # print(f'FPS: {1/delta:.2f}')
        pause = (1 / refresh_rate) - delta
        if pause > 0: time.sleep(pause)
        delta = time.time() - start
        # print(f'FPS: {1/delta:.2f}')

except KeyboardInterrupt:
    cap.release()  # Release the camera resource

cap.release()  # Release the camera resource if break

FPS: 8.06
